In [1]:
from tools.data_reading import CompressedWindowDataset, CocoWindowDataset
import torch
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
from tools import utils
from tools.engine import train_one_epoch, evaluate
from model_zoo.InputInjection import InputInjection
from torch.utils.tensorboard import SummaryWriter

In [2]:
dataset = CocoWindowDataset()

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [3]:
noneC = 0
for i in range(len(dataset)):
    img, spectr, lab = dataset[i]
    if img is None:
        noneC += 1
print(noneC, len(dataset))

1229 2197


In [4]:
train = DataLoader(dataset,batch_size = 1, shuffle=True, collate_fn=utils.collate_fn)

In [10]:
model = InputInjection()
#model.load_state_dict(torch.load(f'inputinjection-train-1.weights'))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'running on {device}')
model.to(device)
writer = SummaryWriter()
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params,lr=0.00003)

running on cpu


In [11]:
evaluate(writer, model, train, device, print_freq=100)

Test:  [   0/2197]  eta: 0:00:25    time: 0.0118  data: 0.0118


/Users/michaelyhuang/ProjectData/python/virtual-environments/ml-venv/lib/python3.9/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/Users/michaelyhuang/ProjectData/python/virtual-environments/ml-venv/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


KeyboardInterrupt: 

In [14]:
model.eval()
for img, spectr, lab in train:
    if img is None:
        continue
    if img[0] is None:
        continue
    img = torch.unsqueeze(img[0], 0).float().to(device)
    spectr = torch.unsqueeze(spectr[0], 0).float().to(device)
    output = model(img, spectr)
    print(output)
    print(lab)
    break
# it's probably not training because the gradient isn't flowing back

[{'boxes': tensor([[  9.2440,   2.7264,  89.3597,  95.1486],
        [105.4982,  89.5808, 127.3933,  95.7915],
        [105.4621,  80.5789, 125.7354,  95.7275],
        [ 68.4715,   5.6651, 122.3567,  94.8436]], device='cuda:0',
       grad_fn=<StackBackward>), 'labels': tensor([1, 1, 1, 1], device='cuda:0'), 'scores': tensor([0.9972, 0.3009, 0.1932, 0.0569], device='cuda:0',
       grad_fn=<IndexBackward>)}]
({'boxes': tensor([[10.,  1., 93., 95.]]), 'labels': tensor([1]), 'image_id': tensor(617), 'area': tensor(617)},)


C:\ProgramData\Miniconda3\lib\site-packages\torch\nn\functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    train_one_epoch(writer, model, optimizer, train, device, epoch, print_freq=100)
    evaluate(writer, model, train, device, print_freq=100)
    torch.save(model.state_dict(), f'inputinjection-train-2.weights')